In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt


plt.rcParams['figure.figsize'] = (20, 8)
plt.rcParams['axes.grid'] = True
                                  
                                  
from dsp import simplify_biquad_filters
import dsp

# Stability Constraints
$$ b_{0j} \le 1 - \gamma $$
$$ b_{1j} - b_{0j} \le 1 - \gamma $$
$$ - b_{1j} \le 1 - \gamma $$

With 

$$ \gamma = 1 - (1 - \epsilon_s)^2 $$


$$ \mathbf{B}\delta \lt \mathbf{b}^{(k)} $$

In [ ]:
def stability_constraints(c, r_max):
    """ Returns the stability constraints matrix B and colum nvector b
    so that

        B*delta < b
    
    in which delta is a optimization variable that updates the filter 
    coefficients.

    Assumed is that delta corresponds to 
    [a10, a11, a12, b10, b11, a20, a21 ... bJ1]

    coeffs: ndarray
        filter coefficients
    r_max : scalar
        the maximum pole radius, which helps establish a stability 
        margin.

    returns B, b
    =======

    B : matrix
    b : column vector
    """
    epsilon_s = 1 - r_max
    b = []
    J = len(c) // 4 # number of filter stages
    for i in range(0, J*4, 4):
        a0, a1, b0, b1 = c[i:i+4]
        b_ = np.c_[(1-epsilon_s)-b0, 
                (1-epsilon_s)-b1+b0, 
                (1-epsilon_s)+b1+b0].T
        b.append(b_)
    b = np.vstack(b)

    J = len(coeffs)
    B = np.zeros((J*3,J*5))
    beta = np.mat('[1 0;-1 1;-1 -1]')

    for i in range(J):
        x = 3+i*5
        y = i*3
        B[y:y+3, x:x+2] = beta
    B = np.matrix(B)
    return B, b